# Recommender

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [0]:
data = spark.sql('SELECT * FROM movielens_ratings')

In [0]:
data.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
 12| 2.0| 0|
 15| 1.0| 0|
 17| 1.0| 0|
 19| 1.0| 0|
 21| 1.0| 0|
 23| 1.0| 0|
 26| 3.0| 0|
 27| 1.0| 0|
 28| 1.0| 0|
 29| 1.0| 0|
 30| 1.0| 0|
 31| 1.0| 0|
 34| 1.0| 0|
 37| 1.0| 0|
 41| 2.0| 0|
+-------+------+------+
only showing top 20 rows

In [0]:
data.describe().show()

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [0]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [0]:
 vc# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [0]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [0]:
predictions.show()

+-------+------+------+------------+
movieId|rating|userId| prediction|
+-------+------+------+------------+
 31| 1.0| 19| 0.9825287|
 31| 1.0| 27| 1.1597517|
 85| 1.0| 4| 3.9298372|
 85| 1.0| 5| 0.4456912|
 85| 5.0| 8| 5.1850953|
 85| 2.0| 20| 1.3649501|
 85| 1.0| 28| 0.2650779|
 65| 2.0| 15| 0.910707|
 65| 1.0| 24| 0.6300031|
 53| 3.0| 20| 2.1454103|
 53| 1.0| 23| -0.11344862|
 78| 1.0| 1| 0.22360995|
 78| 1.0| 4| 0.377413|
 78| 1.0| 8|-0.011558406|
 78| 1.0| 13| 0.61049145|
 78| 1.0| 20| 0.8954493|
 78| 1.0| 22| 0.9585454|
 78| 1.0| 27| 0.734236|
 34| 1.0| 14| 1.8270916|
 34| 1.0| 15| 0.2317043|
+-------+------+------+------------+
only showing top 20 rows

In [0]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 2.025270059576398

The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [0]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [0]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 9| 11|
 18| 11|
 23| 11|
 27| 11|
 35| 11|
 37| 11|
 39| 11|
 41| 11|
 51| 11|
 71| 11|
 80| 11|
 97| 11|
+-------+------+

In [0]:
reccomendations = model.transform(single_user)

In [0]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+------------+
movieId|userId| prediction|
+-------+------+------------+
 71| 11| 2.3313115|
 37| 11| 2.2325523|
 23| 11| 1.467562|
 39| 11| 1.2628573|
 51| 11| 1.128154|
 41| 11| 0.7284756|
 35| 11| 0.22665972|
 97| 11| 0.11294951|
 80| 11|-0.014240559|
 27| 11|-0.026898727|
 9| 11| -0.80148935|
 18| 11| -2.8280342|
+-------+------+------------+

# Great Job!